# TensorRT

In [1]:
import pathlib
import os
import time
import numpy as np
import tensorflow as tf
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import load_model

2024-10-24 19:49:00.342419: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 19:49:00.352145: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 19:49:00.362564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 19:49:00.365663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 19:49:00.374597: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


I0000 00:00:1729813741.212840   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.213016   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.243770   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.243973   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14603864997363321543
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22914203648
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4160016612379630869
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:41:00.0, compute capability: 8.9"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23354408960
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4831840866701438019
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:82:00.0, compute capability: 8.9"
xla_global_id: 2144165316
]


I0000 00:00:1729813741.368871   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369036   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369146   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729813741.369247   18013 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

(8, 6, 1)


### Benchmarking

In [8]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '../best_model/mobilenetv2_best_f1score_fold_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

(10, 5, 0)


### Benchmarking

In [2]:
data_root = pathlib.Path("../data/Monkeypox_Data/Original_Images")    # points to the folder containing the images that will be used for training

# hyperparameters
batch_size = 32         # size of the batch that will be fed to model
img_height = 224        # input image height
img_width = 224         # input image width
test_size = 0.2

# Load dataset without splitting
dataset = tf.keras.utils.image_dataset_from_directory(
    data_root,                                  # loads images from the data_root directory
    image_size=(img_height, img_width),         # resizes all images to (224, 224) pixels
    batch_size=batch_size,                      # set the batch size
    shuffle=True                                # shufle data when loaded
)

# normalization_layer = layers.Rescaling(1./255)
# dataset = dataset.map(lambda x, y: (normalization_layer(x), y))

Found 228 files belonging to 2 classes.


2024-11-12 23:44:19.876237: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:44:19.876630: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:44:20.074795: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### TensorRT Optimization FP16

In [ ]:
params = tf.experimental.tensorrt.ConversionParams(
    precision_mode='FP16'
)
converter = tf.experimental.tensorrt.Converter(
    input_saved_model_dir='../best_model/model1/best_f1score_fold', conversion_params=params
)

model = converter.convert()
# converter.summary()

INFO:tensorflow:Linked TensorRT version: (10, 5, 0)


INFO:tensorflow:Linked TensorRT version: (10, 5, 0)


INFO:tensorflow:Loaded TensorRT version: (10, 5, 0)


INFO:tensorflow:Loaded TensorRT version: (10, 5, 0)
2024-11-13 00:19:39.324218: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:19:39.324439: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:19:39.324540: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2024-11-13 00:19:39.324996: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-13 00:19:39.326599: 

INFO:tensorflow:Clearing prior device assignments in loaded saved model


INFO:tensorflow:Clearing prior device assignments in loaded saved model


INFO:tensorflow:Automatic mixed precision will be used on the whole TensorFlow Graph. This behavior can be deactivated using the environment variable: TF_TRT_EXPERIMENTAL_FEATURES=deactivate_mixed_precision.
More information can be found on: https://www.tensorflow.org/guide/mixed_precision.


INFO:tensorflow:Automatic mixed precision will be used on the whole TensorFlow Graph. This behavior can be deactivated using the environment variable: TF_TRT_EXPERIMENTAL_FEATURES=deactivate_mixed_precision.
More information can be found on: https://www.tensorflow.org/guide/mixed_precision.
2024-11-13 00:19:40.876530: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:19:40.876740: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:19:40.876848: I tensorfl

TRTEngineOP Name                 Device        # Nodes # Inputs      # Outputs     Input DTypes       Output Dtypes      Input Shapes       Output Shapes     
TRTEngineOp_002_000              device:GPU:0  304     1             1             ['float32']        ['float32']        [[-1, 224, 224 ... [[-1, 1]]         

	- AddV2: 63x
	- Cast: 4x
	- Const: 126x
	- Conv2D: 35x
	- DepthwiseConv2dNative: 17x
	- MatMul: 1x
	- Mean: 1x
	- Mul: 18x
	- Pad: 4x
	- Relu6: 35x

[*] Total number of TensorRT engines: 1
[*] % of OPs Converted: 99.35% [304/306]



In [4]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

# Load the SavedModel
saved_model_dir = '../best_model/model1/best_f1score_fold'
optimized_model_dir = '../tensorRT_model/test'

conversion_params = trt.TrtConversionParams(
    precision_mode=trt.TrtPrecisionMode.FP32)

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir='../best_model/model1/mobilenetv2_best_f1score_fold',
    conversion_params=conversion_params)

# Convert the model
converter.convert()
converter.build()
for layer in converter.layers:
    print(f'Layer {layer.name}: {layer.dtype}')

converter.summary()

# Save the optimized model
converter.save(optimized_model_dir)

INFO:tensorflow:Linked TensorRT version: (10, 5, 0)


INFO:tensorflow:Linked TensorRT version: (10, 5, 0)


INFO:tensorflow:Loaded TensorRT version: (10, 5, 0)


INFO:tensorflow:Loaded TensorRT version: (10, 5, 0)
2024-11-13 00:02:07.539197: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:02:07.539553: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-13 00:02:07.539691: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2024-11-13 00:02:07.539821: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-13 00:02:07.540618: 

INFO:tensorflow:Clearing prior device assignments in loaded saved model


INFO:tensorflow:Clearing prior device assignments in loaded saved model


INFO:tensorflow:Automatic mixed precision will be used on the whole TensorFlow Graph. This behavior can be deactivated using the environment variable: TF_TRT_EXPERIMENTAL_FEATURES=deactivate_mixed_precision.
More information can be found on: https://www.tensorflow.org/guide/mixed_precision.


2024-10-24 19:42:51.713773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


RuntimeError: Tensorflow has not been built with TensorRT support.

### TensorRT Optimization INT8

In [4]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


In [3]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf

# Load the SavedModel
saved_model_dir = '../best_model/model1/best_f1score_fold'
optimized_model_dir = '../tensorRT_model/test_INT8'

# Define the conversion parameters
conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
    precision_mode=trt.TrtPrecisionMode.INT8,  # Enable INT8 precision
    max_workspace_size_bytes=8000000000       # 8GB, adjust based on your GPU memory
)

# Create a converter
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_model_dir,
    conversion_params=conversion_params
)

# Function to provide calibration data (generator)
def calibration_input_fn():
    dataset = tf.keras.utils.image_dataset_from_directory(
        "../data/Monkeypox_Data/Original_Images",
        image_size=(224, 224),  # Fixed image size must match model input
        batch_size=156,
        shuffle=True
    )
    for batch, _ in dataset.take(100):  # Higher batch = better calibration?
        batch = batch / 255.0
        yield tf.convert_to_tensor(batch)

# Convert the model and calibrate
converter.convert(calibration_input_fn=calibration_input_fn)

# Explicitly define input shapes for TensorRT during `build()`
print("Building and calibrating the model with fixed input shapes...")
converter.build(input_fn=lambda: [tf.ones((1, 224, 224, 3))])  # Fixed batch size and input shape

# Save the optimized model
converter.save(optimized_model_dir)
print(f"INT8 optimized model saved at {optimized_model_dir}")


INFO:tensorflow:Linked TensorRT version: (10, 5, 0)
INFO:tensorflow:Loaded TensorRT version: (10, 5, 0)


2024-11-12 23:41:17.897050: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:41:17.897432: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:41:17.897536: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2024-11-12 23:41:17.897991: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-12 23:41:17.899085: I external/local_xla/xla/stream_executor/cuda/cuda_e

INFO:tensorflow:Clearing prior device assignments in loaded saved model
INFO:tensorflow:Automatic mixed precision will be used on the whole TensorFlow Graph. This behavior can be deactivated using the environment variable: TF_TRT_EXPERIMENTAL_FEATURES=deactivate_mixed_precision.
More information can be found on: https://www.tensorflow.org/guide/mixed_precision.


2024-11-12 23:41:19.272056: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:41:19.272258: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-12 23:41:19.272364: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 2
2024-11-12 23:41:19.272466: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-11-12 23:41:19.272800: I external/local_xla/xla/stream_executor/cuda/cuda_e

Building and calibrating the model with fixed input shapes...


...

2024-11-12 23:41:20.312224: I tensorflow/compiler/tf2tensorrt/common/utils.cc:120] Linked TensorRT version: 10.5.0
2024-11-12 23:41:20.312250: I tensorflow/compiler/tf2tensorrt/common/utils.cc:122] Loaded TensorRT version: 10.5.0
2024-11-12 23:41:29.391805: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger IElementWiseLayer with inputs StatefulPartitionedCall/sequential_1_1/mobilenetv2_1.00_224_1/bn_Conv1_1/batchnorm/mul_1 and TRTEngineOp_000_000/_tftrt_constant_-0:CONSTANT_output: first input has type Float but second input has type Half.
2024-11-12 23:41:29.392390: W tensorflow/compiler/tf2tensorrt/utils/trt_logger.cc:83] TF-TRT Warning: DefaultLogger IElementWiseLayer with inputs StatefulPartitionedCall/sequential_1_1/mobilenetv2_1.00_224_1/expanded_conv_depthwise_1/depthwise and TRTEngineOp_000_000/_tftrt_constant_-1:CONSTANT_output: first input has type Float but second input has type Half.
2024-11-12 23:41:29.392421: W tensorflow/compiler/tf2

2024-11-12 23:41:31.344408: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 955.51MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-12 23:41:31.375846: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.77GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-12 23:41:31.469860: W tensorflow/compiler/tf2tensorrt/kernels/trt_engine_op.cc:971] TF-TRT Warning: Engine retrieval for input shapes: [[72,224,224,3]] failed. Running native segment for TRTEngineOp_000_000
2024-11-12 23:41:31.509693: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 838.77MiB with fr

INFO:tensorflow:Assets written to: ../tensorRT_model/test_INT8/assets


INFO:tensorflow:Assets written to: ../tensorRT_model/test_INT8/assets


INT8 optimized model saved at ../tensorRT_model/test_INT8
